# Homework 3 Karapet Ghazanjyan

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/spotify.sqlite")
cur = conn.cursor()

In [3]:
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
pd.read_sql(query, conn)

,name
0,albums
1,artists
2,audio_features
3,genres
4,r_albums_artists
5,r_albums_tracks
6,r_artist_genre
7,r_track_artist
8,tracks


In [4]:
conn.text_factory = lambda x: str(x, 'latin1')

In [ ]:
#Top 10 Albums with their artist
pd.read_sql_query('''
SELECT
    albums.name as album_name,
    artists.name as artist_name,
    albums.popularity as albums_popularity
FROM
    albums
JOIN
    r_albums_artists ON albums.id = r_albums_artists.album_id
JOIN
    artists ON r_albums_artists.artist_id = artists.id
ORDER BY
    albums.popularity DESC
limit 10;
''', conn)

,album_name,artist_name,albums_popularity
0,Justice,Justin Bieber,100
1,After Hours,The Weeknd,97
2,EL ÃLTIMO TOUR DEL MUNDO,Bad Bunny,96
3,Fine Line,Harry Styles,95
4,Shoot For The Stars Aim For The Moon,Pop Smoke,94
5,Legends Never Die,Juice WRLD,94
6,Hollywood's Bleeding,Post Malone,93
7,Hot Pink,Doja Cat,93
8,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",Billie Eilish,93
9,Positions,Ariana Grande,93


Just to check on the top 10 albums. We need to be more attentive here as popularity for artists and the albums and the tracks are different.

In [ ]:
#Top 10 Genres of artists
pd.read_sql_query('''
SELECT
    artists.name AS artist_name,
    genres.id AS genre_name,
    artists.popularity as artists_popularity
FROM
    artists
JOIN
    r_artist_genre ON artists.id = r_artist_genre.artist_id
JOIN
    genres ON r_artist_genre.genre_id = genres.id
WHERE
    artists.popularity > 80
ORDER BY
    artists.popularity DESC
limit 10;
''', conn)

,artist_name,genre_name,artists_popularity
0,Drake,canadian hip hop,98
1,Drake,canadian pop,98
2,Drake,hip hop,98
3,Drake,pop rap,98
4,Drake,rap,98
5,Drake,toronto rap,98
6,Bad Bunny,latin,98
7,Bad Bunny,reggaeton,98
8,Bad Bunny,trap latino,98
9,Taylor Swift,pop,97


This shows top 10 genres of different artist by popularity. Here we see that our data is dublicated and for example Drake has the top 6 places, and first two are canadian hip hop and canadian pop, or the 3rd row is hip hop, which I assume are same, and as the app has big variety of tags the data was just downloaded and merged and was not filtered.

In [ ]:
#Top 10 Songs based on popularity with high danceability and energy
pd.read_sql_query('''
SELECT
    tracks.name AS track_name,
    audio_features.danceability,
    audio_features.energy,
    tracks.popularity
FROM
    audio_features
JOIN
    tracks ON audio_features.id = tracks.audio_feature_id
WHERE
    audio_features.danceability > 0.8 AND audio_features.energy > 0.7
ORDER BY
    tracks.popularity DESC
LIMIT 10;
''', conn)

,track_name,danceability,energy,popularity
0,Friday (feat. Mufasa & Hypeman) - Dopamine Re-...,0.824,0.862,94
1,Fiel,0.849,0.701,94
2,Best Friend (feat. Doja Cat),0.840,0.766,91
3,Up,0.868,0.795,90
4,goosebumps,0.841,0.728,89
5,Ride It,0.880,0.751,88
6,No Te Enamores - Remix,0.824,0.837,88
7,SICKO MODE,0.834,0.730,87
8,Godzilla (feat. Juice WRLD),0.808,0.745,87
9,Mi NiÃ±a,0.823,0.821,87


While we checked on the albums and genres, now we look on the top 10 songs with high danceability and energy and a high popularity. This can be used for dance clubs or some parties, where full of dance and energy is required for a GREAT fun.

In [ ]:
#Relationship between track tempo and popularity
pd.read_sql_query('''
SELECT
    tracks.name AS track_name,
    audio_features.tempo,
    tracks.popularity
FROM
    audio_features
JOIN
    tracks ON audio_features.id = tracks.audio_feature_id
WHERE
    tracks.popularity > 50
ORDER BY
    audio_features.tempo DESC;
''', conn)

,track_name,tempo,popularity
0,Mama Cry,220.098999,80
1,Girls and Boys Come out to Play (Instrumental ...,219.826996,52
2,Lazuli,219.445999,54
3,Monsta,219.378006,55
4,When The Beat Drops Out,217.647995,59
...,...,...,...
70496,Pink Noise Sleep Loop (Looped Sleep Pink Noise),0.000000,58
70497,Pouring Rain,0.000000,52
70498,Soft White Noise: Oregon Pacific Coast Ocean W...,0.000000,60
70499,Soothing Brown Waves with the Pouring Rain - L...,0.000000,54


Just wanted to check if tempo can be correlated with popularity, as this can be a key role for the further examinations. Like if songs with high tempo are likely to be more popular, then this affect should be considered when creating a song.

In [ ]:
#The most frequent genres and their average popularity.
pd.read_sql_query('''
SELECT
    genres.id AS genre,
    COUNT(*) AS track_count,
    AVG(tracks.popularity) AS avg_popularity
FROM
    genres
JOIN
    r_artist_genre ON genres.id = r_artist_genre.genre_id
JOIN
    r_track_artist ON r_artist_genre.artist_id = r_track_artist.artist_id
JOIN
    tracks ON r_track_artist.track_id = tracks.id
GROUP BY
    genres.id
ORDER BY
    track_count DESC
LIMIT 10;
''', conn)

,genre,track_count,avg_popularity
0,classical,495467,2.871989
1,classical performance,287065,3.726881
2,adult standards,191882,6.526652
3,rock,136731,17.091208
4,early music,127095,3.525190
5,orchestral performance,119693,3.180971
6,vocal jazz,117062,5.139840
7,orchestra,113422,2.929731
8,classic rock,109292,13.761995
9,lounge,108578,5.269115


Here we see each genre, its track counts and average popularity. This is a reason of many songs having 0 popularity, that means that either they are never rated or maybe in early versions of spotify they did not have that feature (maybe they were outsourcing, like some movie websites from IMDB or etc.) And after getting the data they changed the NAN values to 0. That's why either 0's should be removed with respective rows which will damage the overall output, either some ML model should be applied to fit missing or damaged datas.

In [ ]:
#Explicit tracks that are highly popular.
pd.read_sql_query('''
SELECT
    tracks.name AS track_name,
    tracks.popularity,
    tracks.explicit
FROM
    tracks
JOIN
    audio_features ON tracks.audio_feature_id = audio_features.id
WHERE
    tracks.explicit = 1
ORDER BY
    tracks.popularity DESC
LIMIT 5;
''', conn)

,track_name,popularity,explicit
0,drivers license,100,1
1,Peaches (feat. Daniel Caesar & Giveon),100,1
2,MONTERO (Call Me By Your Name),99,1
3,MONTERO (Call Me By Your Name),99,1
4,good 4 u,99,1


If run for higher limits, it can be seen that when explicit is 1 the popularity of the tracks is high, so this value is also very important. They are highly correlated.

In [ ]:
#Identify instrumental tracks with a high popularity score.
pd.read_sql_query('''
SELECT
    tracks.name AS track_name,
    audio_features.instrumentalness,
    tracks.popularity
FROM
    audio_features
JOIN
    tracks ON audio_features.id = tracks.audio_feature_id
WHERE
    audio_features.instrumentalness > 0.8
ORDER BY
    tracks.popularity DESC
LIMIT 10;
''', conn)

,track_name,instrumentalness,popularity
0,Clean White Noise - Loopable with no fade,1.000,86
1,Can I Call You Tonight?,0.910,82
2,Experience,0.961,80
3,Around the World,0.889,79
4,Levels - Radio Edit,0.828,79
5,YKWIM?,0.893,79
6,White Noise - 145 hz,0.869,78
7,Buttercup,0.810,78
8,Losing Interest,0.865,78
9,Eternal Youth,0.897,78


Also we see here that high instrumentalness can lead to higher than average popularity for tracks, which also means its correlation is higher than average.

In [ ]:
#Identify tracks with high acousticness scores and their popularity.
pd.read_sql_query('''
SELECT
    tracks.name AS track_name,
    audio_features.acousticness,
    tracks.popularity
FROM
    audio_features
JOIN
    tracks ON audio_features.id = tracks.audio_feature_id
WHERE
    audio_features.acousticness > 0.8
ORDER BY
    tracks.popularity DESC
LIMIT 10;
''', conn)

,track_name,acousticness,popularity
0,favorite crime,0.866,94
1,Your Power,0.937,92
2,lovely (with Khalid),0.934,90
3,Memories,0.837,89
4,Arcade,0.818,89
5,Falling,0.839,88
6,Lonely (with benny blanco),0.864,88
7,All of Me,0.922,87
8,If the World Was Ending - feat. Julia Michaels,0.866,87
9,everything i wanted,0.902,87


Same for acousticness, its correlation is higher than average.

In [ ]:
#Identify which artists dominate specific genres in terms of track popularity.
pd.read_sql_query('''
SELECT
    genres.id AS genre,
    artists.name AS artist_name,
    AVG(tracks.popularity) AS avg_popularity,
    COUNT(tracks.id) AS track_count
FROM
    genres
JOIN
    r_artist_genre ON genres.id = r_artist_genre.genre_id
JOIN
    artists ON r_artist_genre.artist_id = artists.id
JOIN
    r_track_artist ON artists.id = r_track_artist.artist_id
JOIN
    tracks ON r_track_artist.track_id = tracks.id
GROUP BY
    genre, artist_name
HAVING
    track_count > 5 -- To filter artists with a reasonable number of tracks in the genre
ORDER BY
    avg_popularity DESC, track_count DESC
LIMIT 10;
''', conn)

,genre,artist_name,avg_popularity,track_count
0,trap latino,Maria Becerra,79.428571,7
1,pop argentino,Maria Becerra,79.428571,7
2,post-teen pop,Harry Styles,75.166667,30
3,pop,Harry Styles,75.166667,30
4,viral rap,CORPSE,74.333333,9
5,dream smp,CORPSE,74.333333,9
6,japanese teen pop,YOASOBI,73.500000,10
7,j-pop,YOASOBI,73.500000,10
8,pop,Billie Eilish,73.152174,46
9,electropop,Billie Eilish,73.152174,46


Here one more time we see dublicated data, which can mislead or misinform us in the future. By this data artist can figure out in which genre they are more popular and can target a specific genre. Either a user can search a specific artist by its specifi genre to find a more popular song.

In [ ]:
#Compare the average energy and danceability of tracks within the most popular albums.
pd.read_sql_query('''
SELECT
    albums.name AS album_name,
    AVG(audio_features.energy) AS avg_energy,
    AVG(audio_features.danceability) AS avg_danceability,
    AVG(tracks.popularity) AS avg_popularity,
    COUNT(tracks.id) AS track_count
FROM
    albums
JOIN
    r_albums_tracks ON albums.id = r_albums_tracks.album_id
JOIN
    tracks ON r_albums_tracks.track_id = tracks.id
JOIN
    audio_features ON tracks.audio_feature_id = audio_features.id
GROUP BY
    album_name
HAVING
    track_count > 5
ORDER BY
    avg_popularity DESC
LIMIT 10;
''', conn)

,album_name,avg_energy,avg_danceability,avg_popularity,track_count
0,EL ÃLTIMO TOUR DEL MUNDO,0.615813,0.714500,79.125000,16
1,YHLQMDLG,0.726400,0.770350,76.350000,20
2,Harry Styles,0.543200,0.510300,76.200000,10
3,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",0.281721,0.583786,75.857143,14
4,RevelaciÃ³n - EP,0.607857,0.756571,75.857143,7
5,THE ALBUM,0.636875,0.697125,75.375000,8
6,Goodbye & Good Riddance,0.566529,0.639824,75.117647,17
7,The Greatest Showman (Original Motion Picture ...,0.496373,0.433545,75.090909,11
8,folklore,0.425062,0.567875,75.062500,16
9,"thank u, next",0.525583,0.668250,75.000000,12


Here we are checking averages for the albums, as there are many tracks in each of them, and by this we can conclude that average energy is from 0.5 to 0.6, and average danceability is more than 0.5 and higher, the album can be very popular, even having less than 10 tracks in it.

In [5]:
#Identify artists with the longest active careers (based on track release dates).
pd.read_sql_query('''
SELECT
    artists.name AS artist_name,
    MIN(DATE(albums.release_date / 1000, 'unixepoch')) AS first_release,
    MAX(DATE(albums.release_date / 1000, 'unixepoch')) AS last_release,
    (julianday(MAX(DATE(albums.release_date / 1000, 'unixepoch'))) -
     julianday(MIN(DATE(albums.release_date / 1000, 'unixepoch')))) AS career_span_days
FROM
    artists
JOIN
    r_albums_artists ON artists.id = r_albums_artists.artist_id
JOIN
    albums ON r_albums_artists.album_id = albums.id
GROUP BY
    artist_name
ORDER BY
    career_span_days DESC
LIMIT 10;
''', conn)

,artist_name,first_release,last_release,career_span_days
0,Johannes Brahms,0000-12-30,2021-03-18,737868.0
1,David Maslanka,0000-12-30,2021-02-05,737827.0
2,Ryan Ross,0000-12-30,2020-12-04,737764.0
3,John Benzer,0000-12-30,2020-11-06,737736.0
4,Brian Merrill,0000-12-30,2020-11-06,737736.0
5,Andy Sealy,0000-12-30,2020-11-06,737736.0
6,James F. Keene,0000-12-30,2019-09-07,737310.0
7,Rachel Horvitz,0000-12-30,2018-07-17,736893.0
8,Cheryl Floyd,0000-12-30,2018-07-17,736893.0
9,Mark Laycock,0000-12-30,2018-03-15,736769.0


It is interesting that first realease is same for most of them, means it was set by default by spotify database team, while for last release the dates are correct, so career span days cannot be calculated correctly. For example "Johannes Brahms' sympony 1 was in 1876 November 4, which is not correct based on this data.

In [6]:
# Identify tracks that are high-energy but underrated in terms of popularity.
pd.read_sql_query('''
SELECT
    tracks.name AS track_name,
    artists.name AS artist_name,
    audio_features.energy,
    tracks.popularity
FROM
    tracks
JOIN
    audio_features ON tracks.audio_feature_id = audio_features.id
JOIN
    r_track_artist ON tracks.id = r_track_artist.track_id
JOIN
    artists ON r_track_artist.artist_id = artists.id
WHERE
    audio_features.energy > 0.8 AND tracks.popularity < 20
ORDER BY
    audio_features.energy DESC, tracks.popularity ASC
LIMIT 10;
''', conn)

,track_name,artist_name,energy,popularity
0,One Angry Ghost,Stuart Turner,1.0,0
1,Moi J'fais Mes Coups En Dessous,Maurice Chevalier,1.0,0
2,Applause; Martha Tilton Returns to Stage - Live,Benny Goodman,1.0,0
3,Applause as Lionel Hampton Enters - Live,Benny Goodman,1.0,0
4,Simple Rain,Winds and Oceans,1.0,0
5,Simple Rain,Focus Ambience,1.0,0
6,Wet Rain,Rain Sound,1.0,0
7,I'm the DJ,The Mindtraveller,1.0,0
8,I'm the DJ,DJ Primo,1.0,0
9,Beating Rain,Sleep Sound Library,1.0,0


This is an example of underrated songs or how energy cannot be correlated with popularity of tracks. This means either we have another problem with wrong data(like there is no rating for a this songs but they are energy full), or they are full energy but very bad popularity and have been forgotten.

In [7]:
#Find artists who appear most frequently in collaborations.
pd.read_sql_query('''
SELECT
    artists.name AS artist_name,
    COUNT(DISTINCT tracks.id) AS collaborative_tracks
FROM
    artists
JOIN
    r_track_artist ON artists.id = r_track_artist.artist_id
JOIN
    tracks ON r_track_artist.track_id = tracks.id
GROUP BY
    artist_name
HAVING
    COUNT(DISTINCT tracks.id) > 5
ORDER BY
    collaborative_tracks DESC
LIMIT 10;
''', conn)

,artist_name,collaborative_tracks
0,Johann Sebastian Bach,42109
1,Wolfgang Amadeus Mozart,33764
2,Ludwig van Beethoven,24999
3,Various Artists,19795
4,FrÃ©dÃ©ric Chopin,18599
5,George Frideric Handel,15078
6,Antonio Vivaldi,14073
7,Giuseppe Verdi,11292
8,Pyotr Ilyich Tchaikovsky,10702
9,Traditional,10625


This are artists who are collobarative and have most colloborative trakcs. Can be used in the future for the marketing department. Just not sure if most of them are still alive.

Hope it was all enough for this assignment, just was not sure if more should have been done. The data will be more interseting if it will be more common for me, like it can be about cars or movies or etc. Never liked to worked with a data where each variable is not specified at all and you should google it to guess which one is what. I am leaving all the outputs as I have queries where 30-40 min runtime was made, and hope in future this big datasets will be filtered before using them.

Have a nice weekend dear Professor